In [1]:
import pandas as pd
import numpy as np
from data import POG4_Dataset
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.feature_selection import RFECV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor
from boruta import BorutaPy

import arfs
import arfs.feature_selection as arfsfs
import arfs.feature_selection.allrelevant as arfsgroot
from arfs.feature_selection import (
    MinRedundancyMaxRelevance,
    GrootCV,
    MissingValueThreshold,
    UniqueValuesThreshold,
    CollinearityThreshold,
    make_fs_summary,
)
from arfs.utils import LightForestClassifier, LightForestRegressor
from arfs.benchmark import highlight_tick, compare_varimp, sklearn_pimp_bench
from arfs.utils import load_data
from arfs.preprocessing import OrdinalEncoderPandas

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor


In [2]:
data = POG4_Dataset()

INFO - Creating XML data
INFO - Creating activity data
INFO - Missing days: 3
INFO - Featurizing time series data
INFO - Creating interactions...


In [3]:
train = data.train[(data.train['date'] >= pd.to_datetime('2020-09-25').date()) & (data.train['date'] <= pd.to_datetime('2021-12-30').date())]


In [4]:

# Using cross-validation so concat the train and test sets
X = train.drop(['sleep_hours', 'date'], axis=1, errors='ignore')
y = train.sleep_hours.fillna(method="ffill")


In [5]:
lgb_kwargs = {"objective": "rmse", "zero_as_missing": False}


basic_fs_pipeline = Pipeline(
    [
        ("missing", arfsfs.MissingValueThreshold(threshold=0.05)),
        ("unique", arfsfs.UniqueValuesThreshold(threshold=1)),
        ("cardinality", arfsfs.CardinalityThreshold(threshold=10)),
        #("collinearity", arfsfs.CollinearityThreshold(threshold=0.99)),
        
    ]
)

X_trans = basic_fs_pipeline.fit_transform(
    X=X, y=y
)

In [6]:
imputer = SimpleImputer(strategy="median")
scaler = RobustScaler()

preprocessor = Pipeline(steps=[("imputer", imputer), ("scaler", scaler)])

X_scaled = pd.DataFrame(preprocessor.fit_transform(X_trans), columns=X_trans.columns)


In [7]:
X_scaled #1518 columns (264 with 0.95)

,AppleStandTime,slp_AppleStandTime_max_hrs_between,slp_AppleStandTime_sum_hrs_between,slp_AppleStandTime_count_hrs_between,slp_AppleStandTime_22_35_00,slp_AppleStandTime_22_40_00,slp_AppleStandTime_22_45_00,slp_AppleStandTime_22_50_00,slp_AppleStandTime_22_55_00,slp_AppleStandTime_23_00_00,...,min_startDate_max_hr,avg_startDate_min_hr,max_startDate_min_hr,min_startDate_min_hr,avg_endDate_max_hr,max_endDate_max_hr,min_endDate_max_hr,avg_endDate_min_hr,max_endDate_min_hr,min_endDate_min_hr
0,-0.175926,0.357143,-0.129032,-0.228571,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.750000,0.666667,0.0,-0.104348,0.142857,0.0,0.636364,0.666667,0.0
1,-0.027778,0.642857,0.043011,-0.514286,2.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.562500,0.333333,0.0,-1.034783,-0.714286,0.0,0.636364,0.333333,0.0
2,2.009259,-0.678571,-0.537634,0.742857,3.0,3.0,3.0,96.000000,24.000000,13.714286,...,-1.0,0.666667,0.000000,0.0,0.626087,0.142857,-1.0,0.636364,0.000000,0.0
3,-0.768519,1.392857,2.580645,-0.971429,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,1.0,-0.430556,0.333333,0.0,-0.921739,-0.714286,1.0,-0.545455,0.333333,0.0
4,0.398148,0.142857,1.376344,0.914286,2.0,0.0,2.0,96.000000,0.000000,0.761905,...,1.0,1.166667,0.333333,0.0,0.313043,0.714286,1.0,1.055944,0.333333,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,1.657407,-2.035714,-0.602151,1.314286,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.562500,0.000000,0.0,-0.243478,-0.714286,0.0,0.454545,0.000000,0.0
63,-1.175926,16.107143,0.817204,-0.857143,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,-6.0,-0.151515,0.000000,0.0,-4.477470,0.571429,-6.0,-0.256198,0.000000,0.0
64,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,4.0,1.916667,0.000000,0.0,4.730435,0.571429,4.0,1.727273,0.000000,0.0
65,-1.527778,-0.607143,-1.655914,-2.400000,1.0,1.0,1.0,26.666667,5.333333,2.285714,...,0.0,6.791667,0.666667,0.0,0.208696,-0.714286,0.0,6.727273,0.666667,0.0


In [18]:
xgb_params = {
    "eta": 0.29930198166014205,
    "alpha": 0.3753739726241434,
    "lmbda": 7.808039770750299,
    "scaler": "robust",
    "imputer": "most_frequent",
    "max_depth": 2,
    "subsample": 0.3134605121170969,
    "n_estimators": 835,
    "learning_rate": 0.03008760669411828,
    "colsample_bytree": 0.313312609952353
}

model_xgb = XGBRegressor(**xgb_params, tree_method="gpu_hist", gpu_id=0, verbosity=0)

In [9]:
# Groota
feat_selector_groota = arfsgroot.BoostAGroota(
    est=model_xgb, iters=100, importance="shap"
)
feat_selector_groota.fit(X_scaled, y)

groota_features = feat_selector_groota.get_feature_names_out()

print(f"The selected features: {groota_features}")
# fig = feat_selector_groota.plot_importance(n_feat_per_inch=5)


BoostaGRoota round:   0%|          | 1/500 [08:47<73:03:44, 527.10s/it]

The selected features: ['AppleStandTime' 'slp_AppleStandTime_max_hrs_between'
 'slp_AppleStandTime_sum_hrs_between'
 'slp_AppleStandTime_startSleep_min_startDate_hr'
 'slp_AppleStandTime_startSleep_max_endDate_hr'
 'slp_AppleExerciseTime_max_hrs_between' 'OxygenSaturation'
 'slp_OxygenSaturation_00_00_00' 'slp_OxygenSaturation_00_55_00'
 'slp_OxygenSaturation_01_10_00' 'slp_OxygenSaturation_01_35_00'
 'slp_ActiveEnergyBurned_23_35_00' 'slp_ActiveEnergyBurned_00_45_00'
 'slp_ActiveEnergyBurned_01_05_00' 'slp_ActiveEnergyBurned_01_30_00'
 'slp_ActiveEnergyBurned_05_25_00' 'slp_ActiveEnergyBurned_07_20_00'
 'slp_BasalEnergyBurned_01_15_00' 'slp_StepCount_sum_hrs_between'
 'slp_DistanceWalkingRunning_02_50_00'
 'slp_DistanceWalkingRunning_04_45_00'
 'slp_DistanceWalkingRunning_07_35_00'
 'slp_EnvironmentalAudioExposure_23_30_00'
 'slp_EnvironmentalAudioExposure_23_35_00'
 'slp_EnvironmentalAudioExposure_23_45_00'
 'slp_EnvironmentalAudioExposure_07_50_00'
 'slp_EnvironmentalAudioExposure_e

In [19]:
# Leshy
feat_selector_leshy = arfsgroot.Leshy(
    model_xgb, max_iter=100, random_state=42, importance="shap"
)
feat_selector_leshy.fit(X_scaled, y)

leshy_features = feat_selector_leshy.get_feature_names_out()


Leshy iteration:  99%|█████████▉| 99/100 [02:51<00:01,  1.74s/it]

All relevant predictors selected in 00:02:51.82
AppleStandTime
slp_AppleStandTime_max_hrs_between
slp_AppleStandTime_sum_hrs_between
slp_AppleStandTime_startSleep_min_startDate_hr
slp_AppleStandTime_startSleep_max_startDate_hr
slp_AppleStandTime_startSleep_min_endDate_hr
slp_AppleStandTime_startSleep_max_endDate_hr
slp_AppleExerciseTime_max_hrs_between
slp_AppleExerciseTime_sum_hrs_between
OxygenSaturation
slp_OxygenSaturation_01_40_00
slp_ActiveEnergyBurned_23_35_00
slp_ActiveEnergyBurned_23_45_00
slp_ActiveEnergyBurned_00_40_00
slp_ActiveEnergyBurned_00_45_00
slp_ActiveEnergyBurned_01_15_00
slp_ActiveEnergyBurned_01_30_00
slp_ActiveEnergyBurned_03_15_00
slp_ActiveEnergyBurned_04_05_00
slp_ActiveEnergyBurned_05_25_00
slp_BasalEnergyBurned_sum_hrs_between
slp_BasalEnergyBurned_01_00_00
slp_HeartRate_01_50_00
slp_HeartRate_02_10_00
slp_HeartRate_07_45_00
RespiratoryRate
slp_RespiratoryRate_07_35_00
slp_RespiratoryRate_07_55_00
slp_RespiratoryRate_endSleep_max_startDate_hr
slp_Respirator

In [21]:

with open('leshy_features.txt', 'w') as f:
    for item in leshy_features:
        print(item)
        f.write("'%s', \n" % item)

AppleStandTime
slp_AppleStandTime_max_hrs_between
slp_AppleStandTime_sum_hrs_between
slp_AppleStandTime_startSleep_min_startDate_hr
slp_AppleStandTime_startSleep_max_startDate_hr
slp_AppleStandTime_startSleep_min_endDate_hr
slp_AppleStandTime_startSleep_max_endDate_hr
slp_AppleExerciseTime_max_hrs_between
slp_AppleExerciseTime_sum_hrs_between
OxygenSaturation
slp_OxygenSaturation_01_40_00
slp_ActiveEnergyBurned_23_35_00
slp_ActiveEnergyBurned_23_45_00
slp_ActiveEnergyBurned_00_40_00
slp_ActiveEnergyBurned_00_45_00
slp_ActiveEnergyBurned_01_15_00
slp_ActiveEnergyBurned_01_30_00
slp_ActiveEnergyBurned_03_15_00
slp_ActiveEnergyBurned_04_05_00
slp_ActiveEnergyBurned_05_25_00
slp_BasalEnergyBurned_sum_hrs_between
slp_BasalEnergyBurned_01_00_00
slp_HeartRate_01_50_00
slp_HeartRate_02_10_00
slp_HeartRate_07_45_00
RespiratoryRate
slp_RespiratoryRate_07_35_00
slp_RespiratoryRate_07_55_00
slp_RespiratoryRate_endSleep_max_startDate_hr
slp_RespiratoryRate_endSleep_max_endDate_hr
day_of_year
min_st

In [11]:

# GrootCV
feat_selector_gcv = arfsgroot.GrootCV(
    objective="rmse", n_iter=100, silent=True
)
feat_selector_gcv.fit(X_scaled, y)

grootcv_features = feat_selector_gcv.get_feature_names_out()

print(f"The selected features: {grootcv_features}")
#fig = feat_selector.plot_importance(n_feat_per_inch=5)

Repeated k-fold: 100%|██████████| 500/500 [01:42<00:00,  4.86it/s]


The selected features: ['RespiratoryRate']


In [13]:
# Combined unique features
combined_features = list(set(groota_features) | set(leshy_features) | set(grootcv_features))

# save to text
with open('combined_features.txt', 'w') as f:
    for item in combined_features:
        print(item)
        f.write("'%s', \n" % item)


slp_OxygenSaturation_00_00_00
slp_ActiveEnergyBurned_07_20_00
slp_EnvironmentalAudioExposure_23_45_00
slp_OxygenSaturation_01_40_00
slp_HeartRate_23_20_00
slp_AppleStandTime_max_hrs_between
slp_OxygenSaturation_00_55_00
slp_ActiveEnergyBurned_04_05_00
min_endDate_max_hr
slp_HeartRate_06_30_00
slp_RespiratoryRate_00_55_00
OxygenSaturation
slp_DistanceWalkingRunning_02_50_00
slp_EnvironmentalAudioExposure_endSleep_max_endDate_hr
slp_RespiratoryRate_01_25_00
slp_AppleStandTime_startSleep_max_startDate_hr
slp_ActiveEnergyBurned_23_35_00
slp_StepCount_sum_hrs_between
AppleStandTime
slp_ActiveEnergyBurned_01_05_00
slp_BasalEnergyBurned_sum_hrs_between
slp_DistanceWalkingRunning_04_45_00
slp_RespiratoryRate_07_55_00
slp_AppleStandTime_startSleep_max_endDate_hr
slp_BasalEnergyBurned_01_15_00
slp_HeartRate_02_50_00
slp_HeartRate_02_15_00
slp_ActiveEnergyBurned_00_45_00
RespiratoryRate
slp_EnvironmentalAudioExposure_07_50_00
slp_ActiveEnergyBurned_01_15_00
slp_ActiveEnergyBurned_23_45_00
slp_Hea

In [20]:

X_groota = X_scaled[groota_features]
X_leshy = X_scaled[leshy_features]
X_gcv = X_scaled[grootcv_features]
X_combined = X_scaled[combined_features]

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

def rmse_cv(model, X, y):
    cv_scores = cross_val_score(model, X, y, cv=2, scoring="neg_mean_squared_error")
    rmse_scores = np.sqrt(-cv_scores)
    avg_rmse = np.mean(rmse_scores)
    return avg_rmse

model_xgb = XGBRegressor(**xgb_params, tree_method="gpu_hist", gpu_id=0, verbosity=0, random_state=42)

print('groota ',rmse_cv(model_xgb, X_groota, y))
print('leshy',rmse_cv(model_xgb, X_leshy, y))
print('gcv',rmse_cv(model_xgb, X_gcv, y))
print('combined',rmse_cv(model_xgb, X_combined, y))

# 15 min interval
# groota  0.9459590260074803
# leshy 0.9474003873683376
# gcv 1.1090371393278202
# combined 0.9806094184326712

groota  0.8817505302361697
leshy 0.8199391049898306
gcv 1.1090371393278202
combined 0.951077979244781
